In [ ]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
from typing import List

In [ ]:
base_url = "https://www.gob.mx"
articles_url = base_url + "/presidencia/es/archivo/articulos?page="
headers = {
    "User-Agent": (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
    )
}
raw_path = Path("raw")
raw_path.mkdir(exist_ok=True, parents=True)


def query(page_to_query):
    response = requests.get(articles_url + str(page_to_query), headers=headers)
    data = [
        BeautifulSoup(d.strip()[21:-2].replace('\\"', '"').replace(r"\/", "/").replace("\\n", ""), "html5lib")
        for d in response.text.split("\n")
        if d.strip()[4:10] == "prensa"
    ]

    return data


def clean_url(url):
    qmark = url.find("?")
    if qmark != -1:
        true_link = url[:qmark]
    else:
        true_link = url
    return base_url + true_link


def get_anchors(documents):
    anchors = []
    for doc in documents:
        anchors.extend([clean_url(a["href"]) for a in doc.find_all("a")])
    return anchors


def get_new_urls(url_list, page) -> List[str]:
    page = page or 1
    url_list = Path(url_list)
    url_list.parent.mkdir(exist_ok=True, parents=True)

    last_fetched = None
    old_urls = []
    if url_list.exists():
        with open(url_list) as readable:
            for old_url in readable:
                old_urls.append(old_url.strip())
        last_fetched = old_urls[0]
        print(f"Last url found {last_fetched}")
    else:
        print("No previous urls found, starting from scratch")

    print(f"Starting fetching from page {page}")

    new_urls = []
    stop_crawling = False
    while not stop_crawling:
        results = query(page)
        if not results:
            print("No more urls, no need to dataset anymore")
            break

        if page % 10 == 0:
            print(f"Querying page {page}")

        for link in get_anchors(results):
            if link == last_fetched:
                print("Found a previously crawled page, no need to dataset anymore")
                stop_crawling = True
                break
            new_urls.append(link)

        page += 1

    all_urls = new_urls + old_urls
    with open(url_list, "w") as writable:
        for url in all_urls:
            writable.write(url + "\n")

    return new_urls


In [ ]:
urls_file = "urls.txt"
new_urls = get_new_urls(urls_file, page=1)

# Imprimir las nuevas URLs encontradas
print("Nuevas URLs encontradas:")
for url in new_urls:
    print(url)

No previous urls found, starting from scratch
Starting fetching from page 1
Querying page 10
Querying page 20
Querying page 30
Querying page 40
Querying page 50
Querying page 60
Querying page 70
Querying page 80
Querying page 90
Querying page 100
Querying page 110
Querying page 120
Querying page 130
Querying page 140
Querying page 150
Querying page 160
Querying page 170
Querying page 180
Querying page 190
Querying page 200
Querying page 210
Querying page 220
Querying page 230
Querying page 240
Querying page 250
Querying page 260
Querying page 270
Querying page 280
Querying page 290
Querying page 300
Querying page 310
Querying page 320
No more urls, no need to dataset anymore
Nuevas URLs encontradas:
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-18-de-septiembre-de-2024
https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-estimulos-a-delegaciones-mexicanas-participantes-en-ju

In [ ]:
new_urls.__len__()

2872

In [ ]:
# Expresión regular para extraer la fecha
date_format = re.compile(r"(?P<day>[0-9]{2}) de (?P<month>[a-z]+) de (?P<year>[0-9]{4})")
meses = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12,
}
#API JINA.AI
prefijo="https://r.jina.ai/"

def parse_document(url: str) -> dict:
    url_completa = prefijo + url
    try:
        # Hacer la solicitud HTTP para obtener el contenido de la página
        response = requests.get(url)
        # Agregar el prefijo de la API a la URL
        respuesta = requests.get(url_completa)

        if response.status_code == 200:
            html_content = response.text
        else:
            print(f"Error al obtener {url}: {response.status_code}")
    except Exception as e:
        print(f"Error al acceder a {url}: {e}")

    try:
        if respuesta.status_code == 200:
            contenido = respuesta.text  # Retornar el código fuente
        else:
            print(f"Error al obtener {url_completa}: {respuesta.status_code}")
    except Exception as e:
        print(f"Error al acceder a {url_completa}: {e}")

    soup = BeautifulSoup(html_content, "html5lib")

    divs = soup.find_all("div", {"class": "pull-left"})

    if len(divs) == 2:
        [article_content] = soup.find_all("div", {"class": "pull-left"})[:1]
    elif len(divs) == 3:
        [article_content] = soup.find_all("div", {"class": "pull-left"})[1:-1]

    # Extraer el título
    title = soup.find("h1").text.strip()

    # Extraer el autor y la fecha
    author, date = [dd.strip() for dd in article_content.find("section").text.split('|')]

    # Parsear la fecha con la expresión regular
    date_info = date_format.match(date).groupdict()

    # Convertir el mes de texto a número
    mes_numero = meses.get(date_info['month'].lower(), 0)  # Usar 0 si no se encuentra el mes
    date_info['month'] = mes_numero

    return {
        'titulo': title,
        'autor': author,
        'dia': date_info['day'],
        'mes': date_info['month'],
        'año': date_info['year'],
        'url': url,
        'contenido': contenido
    }


In [ ]:
urls = ['https://www.gob.mx/presidencia/es/articulos/version-estenografica-visita-del-presidente-y-mensaje-de-la-presidenta-electa-a-las-fuerzas-armadas', 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-10-de-septiembre-de-2024', 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-9-de-septiembre-de-2024', 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-zona-arqueologica-de-ichkabal', 'https://www.gob.mx/presidencia/es/articulos/version-estenografica-inauguracion-del-museo-historico-de-la-ciudad-de-felipe-carrillo-puerto']

In [ ]:
urls= new_urls[:20]

In [ ]:
resultados = []

# Iterar sobre cada URL, hacer la solicitud y procesar el contenido HTML
for url in urls:
  datos = parse_document(url)
  resultados.append(datos)
# Crear un DataFrame con los resultados
mañaneras = pd.DataFrame(resultados)

In [ ]:
mañaneras.shape

(20, 7)

In [ ]:
mañaneras

,titulo,autor,dia,mes,año,url,contenido
0,Versión estenográfica. Conferencia de prensa d...,Presidencia de la República,18,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Conferencia de p...
1,Versión estenográfica. Entrega de estímulos a ...,Presidencia de la República,17,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Entrega de estím...
2,Versión estenográfica. Conferencia de prensa d...,Presidencia de la República,17,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Conferencia de p...
3,Versión estenográfica. Desfile Cívico Militar,Presidencia de la República,16,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Desfile Cívico M...
4,Versión estenográfica. Firma de decreto consti...,Presidencia de la República,15,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Firma de decreto...
5,Versión estenográfica. Inauguración del museo ...,Presidencia de la República,15,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Inauguración del...
6,Versión estenográfica. Bicentenario de la Fede...,Presidencia de la República,14,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Bicentenario de ...
7,Versión estenográfica. 177 Aniversario de la G...,Presidencia de la República,13,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. 177 Aniversario ...
8,Versión estenográfica. Conferencia de prensa d...,Presidencia de la República,13,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Conferencia de p...
9,Versión estenográfica. Conferencia de prensa d...,Presidencia de la República,12,9,2024,https://www.gob.mx/presidencia/es/articulos/ve...,Title: Versión estenográfica. Conferencia de p...


In [ ]:
mañaneras['contenido'][15]

"Title: Versión estenográfica. Inauguración del Museo Histórico de la ciudad de Felipe Carrillo Puerto\n\nURL Source: https://www.gob.mx/presidencia/es/articulos/version-estenografica-inauguracion-del-museo-historico-de-la-ciudad-de-felipe-carrillo-puerto\n\nMarkdown Content:\n**MODERADORA:** Preside este evento, Inauguración del Museo Histórico de la ciudad de Felipe Carrillo Puerto, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.\n\nLo acompañan:\n\nLa presidenta electa de los Estados Unidos Mexicanos, doctora Claudia Sheinbaum Pardo.\n\nLa gobernadora constitucional del estado de Quintana Roo, Mara Lezama Espinosa.\n\nEl secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.\n\nLa secretaria de Medio Ambiente y Recursos Naturales, María Luisa Albores González.\n\nEl director general del Instituto Nacional de Antropología e Historia, Diego Prieto Hernández.\n\nEl secretario de Infraestructura, Comunicaciones 

In [ ]:
texto_ejemplo=mañaneras['contenido'][0]

In [ ]:
def eliminar_seccion(texto):
    # Definir el patrón de búsqueda para "Title" hasta "Content:\n", incluyéndolos
    patron = r'^.*?(?=\*\*)'

    # Reemplazar esa sección por una cadena vacía
    texto_modificado = re.sub(patron, '', texto, flags=re.DOTALL)

    return texto_modificado

In [ ]:
texto_resultante = eliminar_seccion(texto_ejemplo)

In [ ]:
texto_resultante

"**PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR**: Buenos día. Ánimo.\n\n**PREGUNTA:** Ya falta menos.\n\n**PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR:** Ocho. Sí, ¿no? O sea, que las vamos a gozar, van a estar muy buenas estas ocho.\n\nHoy vamos, como todos los miércoles, con la sección de 'Quién es quién en las mentiras de la semana'. Siempre es importante advertir que se trata de una síntesis, una apretada síntesis, un resumen, porque es mucho lo que se miente, son muchas las calumnias, es mucha la guerra sucia, porque estamos en un proceso de transformación, un cambio de régimen que es distinto al gatopardismo y la simulación de que las cosas en apariencia cambian para seguir igual, incluso es distinto un cambio de régimen a un cambio de gobierno.\n\nEntonces, esta es una transformación que está sentando las bases de un sistema democrático, verdadero, que no existía, porque todo era fachada, escenografía, simulación, lo que había era una oligarquía, que es el gobierno de una minoría, pero no

In [ ]:
def extraer_participantes(texto):
    # Definir el patrón para capturar el texto entre "**" y ":**"
    patron = r'\*\*(.*?)\:\*\*'

    # Buscar todas las coincidencias y guardarlas en la lista 'participantes'
    participantes = re.findall(patron, texto)
    if not participantes:
        # Definir un segundo patrón alternativo (por ejemplo, sin los "**")
        patron2 = r'\*\*(.*?)\*\*\:'
        participantes = re.findall(patron2, texto)





    return participantes

In [ ]:
def extraer_participantes(texto):
    # Definir los dos patrones:
    # 1. Patrón para capturar texto entre "**" y ":" (por ejemplo, **NOMBRE:**)
    patron1 = r'\*\*(.*?)\:\*\*'
    # 2. Patrón para capturar texto entre "**" y ":"
    patron2 = r'\*\*(.*?)\*\*\:'

    # Buscar coincidencias del primer patrón
    coincidencias1 = re.findall(patron1, texto)

    # Buscar coincidencias del segundo patrón
    coincidencias2 = re.findall(patron2, texto)


    return participantes

In [ ]:
participantes=extraer_participantes(texto_resultante)

In [ ]:
participantes

['PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESI

In [ ]:
#INUTIL
def eliminar_videos(texto):
    # Paso 1: Eliminar todo lo que se encuentre entre ")**" y "**(", incluyendo los delimitadores
    patron_1 = r'\)\*\*(.*?)\*\*\('
    texto_modificado = re.sub(patron_1, '', texto, flags=re.DOTALL)

    # Paso 2: Eliminar lo que se encuentre entre "**(" y ")**", incluyendo los delimitadores
    #patron_2 = r'\*\*\((.*?)\)\*\*'
    #texto_modificado = re.sub(patron_2, '', texto_modificado, flags=re.DOTALL)

    return texto_modificado

In [ ]:
texto_sin_video=eliminar_videos(texto_resultante)

In [ ]:
texto_sin_video

'**MODERADORA:** Preside esta ceremonia de entrega de estímulos a las delegaciones mexicanas que participaron en los Juegos Olímpicos y Paralímpicos de París 2024, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.\n\nLo acompañan:\n\nLa directora general de la Comisión Nacional de Cultura Física y Deporte, Ana Gabriela Guevara Espinoza.\n\nEl secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.\n\nEl secretario de Marina, almirante José Rafael Ojeda Durán.\n\nLa secretaria de Educación Pública, Leticia Ramírez Amaya.\n\nEl atleta de la disciplina de paratletismo, Juan Pablo Cervantes García.\n\nLa atleta de la disciplina de judo, Prisca Guadalupe Awiti Alcaraz.\n\nAsimismo, damos la bienvenida a deportistas, entrenadores y familiares, así como a los representantes de los medios de comunicación y a quienes no siguen a través de las redes sociales.\n\nTodas y todos ustedes sean bienvenidos.\n\nSean tan amables de

In [ ]:
def extraer_dialogos(texto):
    # Definir el patrón para capturar el texto entre ":**" y "**", incluyendo saltos de línea
    patron = r'\:\*\*(.*?)\*\*'

    # Buscar todas las coincidencias y guardarlas en la lista 'dialogos'
    dialogos = re.findall(patron, texto, flags=re.DOTALL)

    return dialogos

In [ ]:
dialogos=extraer_dialogos(texto_resultante)

In [ ]:
dialogos

[' Preside esta ceremonia de entrega de estímulos a las delegaciones mexicanas que participaron en los Juegos Olímpicos y Paralímpicos de París 2024, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.\n\nLo acompañan:\n\nLa directora general de la Comisión Nacional de Cultura Física y Deporte, Ana Gabriela Guevara Espinoza.\n\nEl secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.\n\nEl secretario de Marina, almirante José Rafael Ojeda Durán.\n\nLa secretaria de Educación Pública, Leticia Ramírez Amaya.\n\nEl atleta de la disciplina de paratletismo, Juan Pablo Cervantes García.\n\nLa atleta de la disciplina de judo, Prisca Guadalupe Awiti Alcaraz.\n\nAsimismo, damos la bienvenida a deportistas, entrenadores y familiares, así como a los representantes de los medios de comunicación y a quienes no siguen a través de las redes sociales.\n\nTodas y todos ustedes sean bienvenidos.\n\nSean tan amables de ocupar sus lu

In [ ]:
def extraer_participantes_y_dialogos(texto):
    # Definir el patrón para capturar tanto los participantes como los diálogos
    patron = r'\*\*(.*?)\:\*\*(.*?)\*\*'

    # Buscar todas las coincidencias
    coincidencias = re.findall(patron, texto, flags=re.DOTALL)

    # Separar las coincidencias en dos listas: una para participantes y otra para diálogos
    participantes2 = [part.strip() for part, _ in coincidencias]
    dialogos2 = [dlg.strip() for _, dlg in coincidencias]

    return participantes, dialogos

In [ ]:
participantes_dialogos=extraer_participantes_y_dialogos(texto_resultante)

In [ ]:
participantes_dialogos

(['MODERADORA',
  'PRISCA GUADALUPE AWITI ALCARAZ, ATLETA DE JUDO',
  'MODERADORA',
  'MODERADORA',
  'ANA GABRIELA GUEVARA ESPINOZA, DIRECTORA GENERAL DE LA COMISIÓN NACIONAL DE CULTURA FÍSICA Y DEPORTE (CONADE)',
  'MODERADORA',
  'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
  'MODERADORA'],
 [' Preside esta ceremonia de entrega de estímulos a las delegaciones mexicanas que participaron en los Juegos Olímpicos y Paralímpicos de París 2024, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.\n\nLo acompañan:\n\nLa directora general de la Comisión Nacional de Cultura Física y Deporte, Ana Gabriela Guevara Espinoza.\n\nEl secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.\n\nEl secretario de Marina, almirante José Rafael Ojeda Durán.\n\nLa secretaria de Educación Pública, Leticia Ramírez Amaya.\n\nEl atleta de la disciplina de paratletismo, Juan Pablo Cervantes García.\n\nLa atleta de la disciplina de judo, Prisca 

In [ ]:
participantes2, dialogos2 = extraer_participantes_y_dialogos(texto_resultante)



In [ ]:
# Mostrar participante 1 y diálogo 1 (índice 0)
print("Participante 1:", participantes2[0])
print("Diálogo 1:", dialogos2[0])

Participante 1: MODERADORA
Diálogo 1:  Preside esta ceremonia de entrega de estímulos a las delegaciones mexicanas que participaron en los Juegos Olímpicos y Paralímpicos de París 2024, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.

Lo acompañan:

La directora general de la Comisión Nacional de Cultura Física y Deporte, Ana Gabriela Guevara Espinoza.

El secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.

El secretario de Marina, almirante José Rafael Ojeda Durán.

La secretaria de Educación Pública, Leticia Ramírez Amaya.

El atleta de la disciplina de paratletismo, Juan Pablo Cervantes García.

La atleta de la disciplina de judo, Prisca Guadalupe Awiti Alcaraz.

Asimismo, damos la bienvenida a deportistas, entrenadores y familiares, así como a los representantes de los medios de comunicación y a quienes no siguen a través de las redes sociales.

Todas y todos ustedes sean bienvenidos.

Sean tan amables 

In [ ]:
participantes

['PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'ANA ELIZABETH GARCÍA VILCHIS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'PREGUNTA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERLOCUTORA',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'JESÚS RAMÍREZ CUEVAS',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
 'INTERVENCIÓN',
 'PRESI

In [ ]:
participante_deseado = "MODERADORA"

# Filtrar los diálogos correspondientes al participante deseado
dialogos_filtrados = [dialogos[i] for i in range(len(participantes)) if participantes[i] == participante_deseado]

# Mostrar los diálogos del participante deseado
if dialogos_filtrados:
    print(f"Diálogos de {participante_deseado}:")
    for idx, dialogo in enumerate(dialogos_filtrados, 1):
        print(f"{idx}. {dialogo}\n")
else:
    print(f"No se encontraron diálogos para {participante_deseado}")

Diálogos de MODERADORA:
1.  Preside esta ceremonia de entrega de estímulos a las delegaciones mexicanas que participaron en los Juegos Olímpicos y Paralímpicos de París 2024, el presidente constitucional de los Estados Unidos Mexicanos, licenciado Andrés Manuel López Obrador.

Lo acompañan:

La directora general de la Comisión Nacional de Cultura Física y Deporte, Ana Gabriela Guevara Espinoza.

El secretario de la Defensa Nacional, general Luis Cresencio Sandoval González.

El secretario de Marina, almirante José Rafael Ojeda Durán.

La secretaria de Educación Pública, Leticia Ramírez Amaya.

El atleta de la disciplina de paratletismo, Juan Pablo Cervantes García.

La atleta de la disciplina de judo, Prisca Guadalupe Awiti Alcaraz.

Asimismo, damos la bienvenida a deportistas, entrenadores y familiares, así como a los representantes de los medios de comunicación y a quienes no siguen a través de las redes sociales.

Todas y todos ustedes sean bienvenidos.

Sean tan amables de ocupar s

In [ ]:
def mostrar_dialogos_de_participante(participante_deseado, participantes, dialogos):
    # Filtrar y mostrar los diálogos correspondientes al participante deseado
    dialogos_de_participante = [dialogos[i] for i in range(len(participantes)) if participantes[i] == participante_deseado]

    if dialogos_de_participante:
        print(f"Diálogos de {participante_deseado}:")
        for idx, dialogo in enumerate(dialogos_de_participante, 1):
            print(f"{idx}. {dialogo}\n")
    else:
        print(f"No se encontraron diálogos para {participante_deseado}")

In [ ]:
participantes, dialogos = extraer_participantes_y_dialogos(texto_resultante)

# Mostrar todos los diálogos de 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR'
mostrar_dialogos_de_participante("PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR", participantes2, dialogos2)

IndexError: list index out of range

In [ ]:
dialogos2.__len__()

8

In [ ]:
participantes2.__len__()

8

In [ ]:
participantes.__len__()

115

In [ ]:
dialogos.__len__()

114